# Fase 3: Data Preparation

## Importar librerias

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from collections import Counter

## Importar los parquet desde el drive

Para comenzar, montaremos el drive que contiene los archivos parquet que nos proporcionará los datos para el analisis de los datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# # Ruta de tus archivos
# archivos = glob.glob('/content/drive/MyDrive/Anime Dataset/*.parquet')

# # Ordenar archivos
# archivos = sorted(archivos)
# print("cargando")

# # Cargar cada archivo en una variable con un nombre derivado del archivo
# for ruta in archivos:
#     # Extraer el nombre del archivo sin la ruta y la extensión
#     nombre_archivo = ruta.split('/')[-1]
#     nombre_variable = nombre_archivo.replace('.parquet', '')

#     # Cargar el archivo en la variable
#     globals()[nombre_variable] = pd.read_parquet(ruta)
#     print(f"'{nombre_variable}' cargado con {len(globals()[nombre_variable])} filas")


In [4]:
# Ruta base de tus archivos en Google Drive
base_path = '/content/drive/MyDrive/Anime Dataset/' # Asegúrate de que esta ruta sea correcta

# Lista de los nombres de archivos específicos que quieres cargar
file_names = ['anime_dataset_2023.parquet', 'users_score_2023.parquet', 'users_details_2023.parquet'] # Corregí la lista para incluir solo los nombres de archivo únicos

print("cargando")

# Cargar cada archivo en una variable con un nombre derivado del archivo
for file_name in file_names:
    ruta = base_path + file_name
    # Extraer el nombre del archivo sin la extensión para usar como nombre de variable
    nombre_variable = file_name.replace('.parquet', '')

    # Cargar el archivo en la variable
    try:
        globals()[nombre_variable] = pd.read_parquet(ruta)
        print(f"'{nombre_variable}' cargado con {len(globals()[nombre_variable])} filas")
    except FileNotFoundError:
        print(f"Error: El archivo '{file_name}' no fue encontrado en la ruta especificada.")
    except Exception as e:
        print(f"Error al cargar el archivo '{file_name}': {e}")


cargando
'anime_dataset_2023' cargado con 24905 filas
'users_score_2023' cargado con 24325191 filas
'users_details_2023' cargado con 731290 filas


## Vemos con que columnas trabajaremos

Decidimos en este caso no trabajar con el Dataset de **user_details_2023**, **final_animedata**,

## Creación de la copia de los DataFrame

Para poder comenzar a trabajar en la preparación de los datos sin transformar los conjuntos de datos iniciales, vamos a crear copias de los DataFrame. Esto nos permite poder realizar los cambios y transformaciones sin riesgos de perder la información original.

In [5]:
anime_dataset = anime_dataset_2023.copy()
users_detail = users_details_2023.copy()
users_score= users_score_2023.copy()

## Verificación de columnas y tipo de datos

Ahora verificaremos que columnas estan dentro de nuestros DataFrame y el tipo de dato de cada una.

##AnimeDataset

In [6]:
 anime_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   anime_id      24905 non-null  int32   
 1   Name          24905 non-null  object  
 2   English name  24905 non-null  category
 3   Other name    24905 non-null  object  
 4   Score         24905 non-null  category
 5   Genres        24905 non-null  category
 6   Synopsis      24905 non-null  object  
 7   Type          24905 non-null  category
 8   Episodes      24905 non-null  category
 9   Aired         24905 non-null  object  
 10  Premiered     24905 non-null  category
 11  Status        24905 non-null  category
 12  Producers     24905 non-null  category
 13  Licensors     24905 non-null  category
 14  Studios       24905 non-null  category
 15  Source        24905 non-null  category
 16  Duration      24905 non-null  category
 17  Rating        24905 non-null  category
 18  Rank  

In [7]:
# Inicializar un objeto Counter
conteo_generos = Counter()

# Iterar a través de la columna 'Genres'
for lista_generos in anime_dataset['Genres']:
    # Dividir la cadena de géneros por coma y espacio, luego actualizar el contador
    for genero in lista_generos.split(', '):
        conteo_generos[genero] += 1

# Imprimir el conteo de géneros
for genero, cantidad in conteo_generos.most_common():
    print(f"{genero}: {cantidad}")

Comedy: 7142
Fantasy: 5306
UNKNOWN: 4929
Action: 4730
Adventure: 3842
Sci-Fi: 3091
Drama: 2836
Romance: 2063
Slice of Life: 1755
Supernatural: 1494
Hentai: 1486
Mystery: 847
Avant Garde: 804
Ecchi: 795
Sports: 771
Horror: 534
Suspense: 242
Award Winning: 241
Boys Love: 169
Gourmet: 146
Girls Love: 113
Erotica: 54


###Eliminacion de columnas innecesesarias


In [8]:
columns_to_drop = ['English name', 'Other name', 'Producers', 'Licensors', 'Studios', 'Image URL']
anime_dataset = anime_dataset.drop(columns=columns_to_drop)

# Para verificar las columnas restantes
print(anime_dataset.columns)

Index(['anime_id', 'Name', 'Score', 'Genres', 'Synopsis', 'Type', 'Episodes',
       'Aired', 'Premiered', 'Status', 'Source', 'Duration', 'Rating', 'Rank',
       'Popularity', 'Favorites', 'Scored By', 'Members'],
      dtype='object')


In [9]:
anime_dataset

,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",UNKNOWN,Finished Airing,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978
2,6,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252
3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",summer 2002,Finished Airing,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931
4,8,Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",fall 2004,Finished Airing,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",UNKNOWN,Not yet aired,Web manga,Unknown,PG-13 - Teens 13 or older,UNKNOWN,24723,0,UNKNOWN,0
24901,55732,Bu Xing Si: Yuan Qi,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",UNKNOWN,Not yet aired,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0
24902,55733,Di Yi Xulie,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",UNKNOWN,Finished Airing,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",UNKNOWN,Finished Airing,Original,3 min,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0


###Modificacion de datos


In [10]:
# Reemplazar 'UNKNOWN' por 0 en la columna 'Episodes'
anime_dataset['Episodes'] = anime_dataset['Episodes'].replace('UNKNOWN', 0)

# Convertir la columna 'Episodes' a tipo entero
# Usamos errors='coerce' para convertir cualquier valor que no se pueda transformar a numérico en NaN
# Luego rellenamos los NaN (si los hay) con 0 antes de convertir a int
anime_dataset['Episodes'] = pd.to_numeric(anime_dataset['Episodes'], errors='coerce').fillna(0).astype(int)

# Verificar los tipos de datos después de la conversión
print(anime_dataset['Episodes'].dtype)
score_counts_sorted_by_score = anime_dataset['Episodes'].value_counts().sort_index(ascending=True)
print(score_counts_sorted_by_score)

int64
Episodes
0         611
1       11532
2        1528
3         712
4         553
        ...  
1565        1
1664        1
1787        1
1818        1
3057        1
Name: count, Length: 252, dtype: int64


/tmp/ipython-input-2202945054.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  anime_dataset['Episodes'] = anime_dataset['Episodes'].replace('UNKNOWN', 0)


Cambiamos los datos UNKNOWN dentro de episodios a 0 y cambiamos el tipo a numerico

In [11]:
# Reemplazar 'UNKNOWN' con False y otros valores con True en la columna 'Premiered'
# Usamos .loc para evitar SettingWithCopyWarning si el DataFrame es una vista
anime_dataset.loc[:, 'Premiered'] = anime_dataset['Premiered'] != 'UNKNOWN'

# Verificar los primeros valores y los tipos de datos después de la conversión
print(anime_dataset['Premiered'].head())
print(anime_dataset['Premiered'].dtype)

0     True
1    False
2     True
3     True
4     True
Name: Premiered, dtype: bool
bool


/tmp/ipython-input-3917092555.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ True False  True ... False False False]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  anime_dataset.loc[:, 'Premiered'] = anime_dataset['Premiered'] != 'UNKNOWN'


Cambiamos la columna premiados por true si tiene posee algún premio y por false si dice UNKNOWN

In [12]:
anime_dataset['Rank'] = anime_dataset['Rank'].replace('UNKNOWN', 0)

anime_dataset['Rank'] = pd.to_numeric(anime_dataset['Rank'], errors='coerce').fillna(0).astype(int)

num_rank = anime_dataset['Rank'].value_counts().sort_index(ascending=True)
print(num_rank)

Rank
0        4799
1           1
2           1
3           1
4           1
         ... 
20098       2
20100       2
20101       2
20103       1
20104       2
Name: count, Length: 15197, dtype: int64


Modificación de la columna rank, se cambiaron los datos UNKNOWN por 0 y se cambio el tipo de category a int para una posible regresión

##users_detail

In [13]:
users_detail

,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.300003,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.599998,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.800003,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0
3,9,Arcane,NaN,NaN,NaN,2004-12-05T00:00:00+00:00,30.000000,7.71,5.0,54.0,4.0,3.0,0.0,66.0,0.0,1817.0
4,18,Mad,NaN,NaN,NaN,2005-01-03T00:00:00+00:00,52.000000,6.27,1.0,114.0,10.0,5.0,23.0,153.0,42.0,3038.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731285,1291083,Dolopa,NaN,NaN,NaN,2012-05-06T00:00:00+00:00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731286,1291085,alenrobnik,NaN,NaN,NaN,2012-05-06T00:00:00+00:00,21.400000,8.22,16.0,58.0,9.0,1.0,42.0,126.0,0.0,1239.0
731287,1291087,Oblongata,NaN,1993-01-30T00:00:00+00:00,NaN,2012-05-06T00:00:00+00:00,51.299999,7.53,38.0,175.0,0.0,9.0,211.0,433.0,2.0,3010.0
731288,1291091,Etnota,NaN,NaN,NaN,2012-05-06T00:00:00+00:00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
users_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731290 entries, 0 to 731289
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Mal ID            731290 non-null  int32   
 1   Username          731289 non-null  object  
 2   Gender            224383 non-null  category
 3   Birthday          168068 non-null  category
 4   Location          152805 non-null  category
 5   Joined            731290 non-null  category
 6   Days Watched      731282 non-null  float32 
 7   Mean Score        731282 non-null  float32 
 8   Watching          731282 non-null  float32 
 9   Completed         731282 non-null  float32 
 10  On Hold           731282 non-null  float32 
 11  Dropped           731282 non-null  float32 
 12  Plan to Watch     731282 non-null  float32 
 13  Total Entries     731282 non-null  float32 
 14  Rewatched         731282 non-null  float32 
 15  Episodes Watched  731282 non-null  float32 
dtypes:

###Eliminación de columnas

In [15]:
columns_a_eliminar = ['Location', 'Joined', 'Birthday', 'Days Watched', 'Total Entries', 'On Hold', 'Dropped', 'Watching', 'Completed', 'Plan to Watch', 'Rewatched', 'Episodes Watched', 'Mean Score']
users_detail = users_detail.drop(columns=columns_a_eliminar)

# Para verificar las columnas restantes
print(users_detail.columns)

Index(['Mal ID', 'Username', 'Gender'], dtype='object')


##UserScore


In [16]:
users_score

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8
...,...,...,...,...,...
24325186,1291087,Oblongata,10611,R-15,3
24325187,1291087,Oblongata,174,Tenjou Tenge,6
24325188,1291097,JuunanaSai,1535,Death Note,9
24325189,1291097,JuunanaSai,226,Elfen Lied,10


In [17]:
users_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 5 columns):
 #   Column       Dtype   
---  ------       -----   
 0   user_id      int32   
 1   Username     category
 2   anime_id     int32   
 3   Anime Title  category
 4   rating       int32   
dtypes: category(2), int32(3)
memory usage: 428.3 MB


In [18]:
rating_counts = users_score['rating'].value_counts()
print(rating_counts)

rating
8     6060484
7     5452152
9     4429914
10    3210021
6     2766482
5     1379480
4      562822
3      233675
2      132092
1       98069
Name: count, dtype: int64


In [19]:
print(users_score.isnull().sum())

user_id          0
Username       232
anime_id         0
Anime Title      0
rating           0
dtype: int64


###Eliminación de datos nulos

In [20]:
# Eliminar filas con valores nulos en la columna 'Username'
users_score.dropna(subset=['Username'], inplace=True)

# Verificar si quedan valores nulos en la columna 'Username'
print(users_score['Username'].isnull().sum())

0


In [21]:
anime_dataset.head(2)

,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26,"Apr 3, 1998 to Apr 24, 1999",True,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193.0,1771505
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1,"Sep 1, 2001",False,Finished Airing,Original,1 hr 55 min,R - 17+ (violence & profanity),189,602,1448,206248.0,360978


In [22]:
users_detail.head(2)

,Mal ID,Username,Gender
0,1,Xinil,Male
1,3,Aokaado,Male


In [23]:
users_score.head(2)

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7


#Union de tablas

In [24]:
# Realizar el merge de users_detail y users_score
# Asumimos que 'Mal ID' en users_detail corresponde a 'user_id' en users_score
final_users = pd.merge(users_detail, users_score, left_on='Mal ID', right_on='user_id', how='inner')

# Mostrar las primeras filas y la información del nuevo DataFrame para verificar
display(final_users.head())
print(final_users.info())

,Mal ID,Username_x,Gender,user_id,Username_y,anime_id,Anime Title,rating
0,1,Xinil,Male,1,Xinil,21,One Piece,9
1,1,Xinil,Male,1,Xinil,48,.hack//Sign,7
2,1,Xinil,Male,1,Xinil,320,A Kite,5
3,1,Xinil,Male,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,Male,1,Xinil,304,Aa! Megami-sama! Movie,8


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23803547 entries, 0 to 23803546
Data columns (total 8 columns):
 #   Column       Dtype   
---  ------       -----   
 0   Mal ID       int32   
 1   Username_x   object  
 2   Gender       category
 3   user_id      int32   
 4   Username_y   category
 5   anime_id     int32   
 6   Anime Title  category
 7   rating       int32   
dtypes: category(3), int32(4), object(1)
memory usage: 714.5+ MB
None


In [25]:
columns_a_eliminar = ['Mal ID', 'Username_y']
final_users = final_users.drop(columns=columns_a_eliminar)

# Para verificar las columnas restantes
print(final_users.columns)

Index(['Username_x', 'Gender', 'user_id', 'anime_id', 'Anime Title', 'rating'], dtype='object')


In [27]:
final_anime_dataset = pd.merge(anime_dataset, final_users, left_on='anime_id', right_on='anime_id', how='inner')

# Eliminar la columna 'Name' ya que 'Anime Title' es similar
final_anime_dataset = final_anime_dataset.drop(columns=['Name'])

# Reordenar las columnas para poner las de usuario primero
# Identificar las columnas de usuario (excluyendo 'anime_id' que es la clave de unión)
user_columns = [col for col in final_users.columns if col != 'anime_id']
# Identificar las columnas de anime (excluyendo 'anime_id' y 'Name' que eliminamos)
anime_columns = [col for col in anime_dataset.columns if col != 'anime_id' and col != 'Name']

# Crear la nueva lista de orden de columnas: columnas de usuario + 'anime_id' + columnas de anime
# Aseguramos que 'Anime Title' esté en las columnas de usuario si es que viene de ahí
# O la manejamos explícitamente si es necesario ajustar el orden después de eliminar 'Name'
# Basándonos en el head anterior, 'Anime Title' viene de final_users y ya está en user_columns
new_column_order = user_columns + ['anime_id'] + anime_columns

# Aplicar el nuevo orden de columnas al DataFrame
final_anime_dataset = final_anime_dataset[new_column_order]


# Mostrar las primeras filas y la información del nuevo DataFrame para verificar
display(final_anime_dataset.head())
print(final_anime_dataset.info())

,Username_x,Gender,user_id,Anime Title,rating,anime_id,Score,Genres,Synopsis,Type,...,Premiered,Status,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members
0,Xinil,Male,1,Cowboy Bebop,10,1,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,...,True,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193.0,1771505
1,Crystal,Female,4,Cowboy Bebop,8,1,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,...,True,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193.0,1771505
2,vondur,Male,20,Cowboy Bebop,9,1,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,...,True,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193.0,1771505
3,Amuro,NaN,23,Cowboy Bebop,9,1,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,...,True,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193.0,1771505
4,kei-clone,Male,47,Cowboy Bebop,7,1,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,...,True,Finished Airing,Original,24 min per ep,R - 17+ (violence & profanity),41,43,78525,914193.0,1771505


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23803016 entries, 0 to 23803015
Data columns (total 22 columns):
 #   Column       Dtype   
---  ------       -----   
 0   Username_x   object  
 1   Gender       category
 2   user_id      int32   
 3   Anime Title  category
 4   rating       int32   
 5   anime_id     int32   
 6   Score        category
 7   Genres       category
 8   Synopsis     object  
 9   Type         category
 10  Episodes     int64   
 11  Aired        object  
 12  Premiered    bool    
 13  Status       category
 14  Source       category
 15  Duration     category
 16  Rating       category
 17  Rank         int64   
 18  Popularity   int32   
 19  Favorites    int32   
 20  Scored By    category
 21  Members      int32   
dtypes: bool(1), category(10), int32(6), int64(2), object(3)
memory usage: 1.8+ GB
None
